Implement collaborative filtering to build a recommender for movies.

In [2]:
# Initial imports:

import numpy as np
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *

As in notes, notattion is:

$r(i, j) = 1$ if user $j$ rated movie $i$, else $0$

$y(i, j) :=$ user $j$'s rating of movie $i$, if $r(i, j) == 1$

$w^{(j)}$: parameters for user $j$

$b^{(j)}$: parameter for user $j$

$x^{(i)}$: feature ratings for movie $i$

$n_u$ or num_users

$n_m$ or num_movies

$n$ or num_features

$X$: matrix of vectors $x^{(i)}$

$W$: matrix of vectors $w^{(j)}$

$b$: vector of bias parameters $b^{(j)}$

$R$: matrix of elements $r(i, j)$



In [3]:
# Load data:
X, W, b, num_movies, num_features, num_users = load_precalc_params_small()
Y, R = load_ratings_small()

# Brief overview
print(f"""
Y: {Y.shape}
R: {R.shape}
X: {X.shape}
W: {W.shape}
b: {b.shape}
num_movies: {num_movies}
num_features: {num_features}
num_users: {num_users}
""")


Y: (4778, 443)
R: (4778, 443)
X: (4778, 10)
W: (443, 10)
b: (1, 443)
num_movies: 4778
num_features: 10
num_users: 443



In [4]:
# From these base data, get average rating:
tsmean = np.mean(Y[0, R[0, :].astype(bool)])
print(f"Average rating for movie 1: {tsmean:0.3f} / 5")

Average rating for movie 1: 3.400 / 5


See notes for the cofi algorithm, it's a doozy.

Do note that the summation over $(i, j): r(i, j) = 1$ can be expressed as summation over the elements times $r(i, j)$, which filters out the 0-values anyway.

Initially, we start with a for-loop:

In [5]:
def cofi_cost_func_loop(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering.
    Args:
        X (ndarray (num_movies, num_features)): matrix of item features
        W (ndarray (num_users, num_features)): matrix of user parameters
        b (ndarray (1, num_users): vector of user parameters
        Y (ndarray (num_movies, num_users)): matrix of user ratings
        R (ndarray (num_movies, num_users)): matrix where R(i, j) = 1 if user j has rated movie i
        lambda_ (float): regularization parameter
        
    Returns:
        J (float): Cost
    """
    
    nm, nu = Y.shape
    J = 0
    
    for j in range(nu):
        w = W[j, :]
        bj = b[0, j]
        
        for i in range(nm):
            x = X[i, :]
            r = R[i, j]
            y = Y[i, j]
            
            J += np.multiply(r, (np.dot(w, x) + bj - y) ** 2)
    
    J += lambda_ * (np.sum(np.power(W, 2)) + np.sum(np.power(X, 2)))
    J /= 2
    
    return J

In [7]:
# Test
nu_r = 4
nm_r = 5
n_r = 3
X_r = X[:nm_r, :n_r]
W_r = W[:nu_r, :n_r]
b_r = b[0, :nu_r].reshape(1, -1)
Y_r = Y[:nm_r, :nu_r]
R_r = R[:nm_r, :nu_r]

# Without reg
J_r = cofi_cost_func_loop(X_r, W_r, b_r, Y_r, R_r, 0)
print(J_r)
# Expect 13.67

# With reg
J_r = cofi_cost_func_loop(X_r, W_r, b_r, Y_r, R_r, 1.5)
print(J_r)
# Expect 28.09

13.670725805579915
28.09383799145902


Obviously, fuck loops. Try for vectorized!

In [16]:
def cofi_cost_func(X, W, b, Y, R, lambda_):
    """
    Returns the cost for the content-based filtering.
    Args:
        X (ndarray (num_movies, num_features)): matrix of item features
        W (ndarray (num_users, num_features)): matrix of user parameters
        b (ndarray (1, num_users): vector of user parameters
        Y (ndarray (num_movies, num_users)): matrix of user ratings
        R (ndarray (num_movies, num_users)): matrix where R(i, j) = 1 if user j has rated movie i
        lambda_ (float): regularization parameter
        
    Returns:
        J (float): Cost
    """
    
    # Non-reg
    j = np.sum(np.power((np.matmul(X, np.transpose(W)) + b - Y) * R, 2))
    
    # Reg
    J = 0.5 * (j + lambda_ * (np.sum(np.power(X, 2)) + np.sum(np.power(W, 2))))
    
    return J
    

In [18]:
print(cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 0))
print(cofi_cost_func(X_r, W_r, b_r, Y_r, R_r, 1.5))

13.670725805579915
28.09383799145902
